In [ ]:
ls

In [ ]:
cd run_rasqual/

# IMPUTATION and PHASING (method 1: IMPUTE2)
Not finished. We choose another method (see method 2 below) to do imputation.
For next step scripts and other useful scripts, see [here](https://github.com/JoniColeman/gwas_scripts/blob/master/README.md).

In [ ]:
! wget https://mathgen.stats.ox.ac.uk/impute/impute_v2.3.2_x86_64_dynamic.tgz

In [ ]:
! tar -zxvf impute_v2.3.2_x86_64_dynamic.tgz

In [ ]:
! cd impute_v2.3.2_x86_64_dynamic/; mkdir reference;cd reference

In [ ]:
! wget https://mathgen.stats.ox.ac.uk/impute/1000GP_Phase3.tgz

In [ ]:
! tar zxvf 1000GP_Phase3.tgz

## Prepare data to run IMPUTE2

data obtained from Siwei Zhang via slack:

In [ ]:
ls ./data/

convert plink data format to vcf format

In [ ]:
! cd data/;module load plink; plink --bfile plink --recode vcf --out genotype_plink

convert vcf format to gens for IMPUTE2

In [ ]:
! cd ..; wget https://mathgen.stats.ox.ac.uk/impute/scripts/vcf2impute_gen

In [ ]:
! ./vcf2impute_gen -vcf data/genotype_plink.vcf -gen data/genotype_plink.gens

## run IMPUTE2

# IMPUTATION and PHASING (method 2)
We use [Michigan imputation server](https://imputationserver.sph.umich.edu/index.html#!pages/home).
They require vcf format as input. need to split by chromosome. It seems the plink files sent by Siwei were hg18. So we need to first convert hg18 to b37 vcf files, then split by chromosome, sort and gzip following their instructions [here](https://imputationserver.sph.umich.edu/index.html#!pages/help).

To liftover VCF files, use [ensemble API](https://useast.ensembl.org/Homo_sapiens/Tools/AssemblyConverter?db=core). To use this online tool, file can't exceed 50 MB, so need to split by chr first, merge into two subfiles, and liftover each subfile). See more options for liftover vcf files [here](https://www.biostars.org/p/138042/).

In [ ]:
! cd /home/simingz/run_rasqual/data;mkdir vcf_split_by_chr/
! module load vcftools; seq 1 22 | xargs -n1 -P4 -I {} vcftools --vcf genotype_plink.vcf --chr {} --recode --recode-INFO-all --out vcf_split_by_chr/genotype_plink.chr{}

In [ ]:
! cd vcf_split_by_chr/; for i in `seq 1 22`; do vcf-sort genotype_plink.chr${i}.recode.vcf | bgzip -c > genotype_plink.chr${i}.recode.vcf.gz; done

In [ ]:
vcf-concat ge*chr?.*.vcf.gz > chr_batch1.vcf
vcf-concat ge*chr??.*.vcf.gz > chr_batch2.vcf

In [ ]:
cd data/vcf_split_by_chr/

upload liftover vcf files from local computer. 

In [ ]:
! mkdir b37_bychr; seq 1 9 | xargs -n1 -P4 -I {} vcftools --vcf chr_batch1.b37.vcf --chr {} --recode --recode-INFO-all --out b37_bychr/genotype_plink.chr{}.b37 
! seq 10 22 | xargs -n1 -P4 -I {} vcftools --vcf chr_batch2.b37.vcf --chr {} --recode --recode-INFO-all --out b37_bychr/genotype_plink.chr{}.b37

In [ ]:
! cd b37_chr/; for i in `seq 1 22`; do vcf-sort genotype_plink.chr${i}.b37.recode.vcf | bgzip -c > genotype_plink.chr${i}.recode.b37.vcf.gz; done

upload vcf.gz files to [Michigan imputation server](https://imputationserver.sph.umich.edu/index.html#!pages/home). Finish running. the password to extract file is: DK*9uja5OiJv0U

In [ ]:
cd /home/simingz/run_rasqual/phased_data

# PREPARE RASQUAL INPUT

This [paper](https://www.nature.com/articles/s41588-017-0005-8) used rasqual to find caQTL using atac seq data. They processed their ATAC seq data (N=24) as follows:
> **Peak calling**
We used MACS2 v2.1.1 (Zhang et al. 2008) to call ATAC-seq peaks for individual samples
with parameters ‘--nomodel --shift -25 --extsize 50 -q 0.01’. We defined a consensus set of
peaks as regions in which peaks overlapped in at least 3 samples. At regions of overlap, the
consensus peak was defined as the union of overlapping peaks. This resulted in 381,323
peaks, with 98% of peaks ranging from 82 - 1191 base pairs.

> (gene QTLs)
We ran RASQUAL18 with the option --no-posterior-update for each of 35,033 genes, testing SNPs and indels (MAF > 0.05; INFO > 0.8) within 500 kb of the gene transcription start site. For each gene we applied Bonferroni correction to the P values according to the number of independent tests estimated by EigenMT44. We ran RASQUAL with the option --random-permutation to call QTLs with permuted sample labels, and noted the minimum EigenMT-corrected P value per gene. To determine the FDR for eQTL discovery at a given gene, we computed (#permuted data min p values < p)/(#real data min p values < p), where p is the minimum P value among SNPs for the gene in question. For QTL calling with FastQTL, we used CQN-transformed gene expression (cqn v5.0.2)44 with a cis-window of 500 kb, including 20 PCs as covariates. We determined tissue-specific IPSDSN genes (not in GTEx) by using a protocol described for the HIPSCI project8.

> **ATAC QTLs calling**
As we did for gene expression, we used featureCounts v1.5.0 to count fragments
overlapping consensus ATAC-seq peaks and ASEReadCounter to count allele-specific
reads at SNPs (and not indels) within peaks. We ran RASQUAL separately for each of
381,323 peaks, passing in VCF lines for SNPs and indels (MAF > 0.05, INFO > 0.8) within 1
kb of the center of the peak. Since >99.9% of peaks were less than 2 kb in size, this meant
that we tested effectively all SNPs within peaks. As we did when calling eQTLs, we ran
RASQUAL with the --random-permutation option for every gene, and determined FDR as
described above. Note that in this case we used Bonferroni correction based on the number
of SNPs tested, without using EigenMT, due to the small size of the windows tested. With
this procedure we obtained 6,318 ATAC peaks with a cis-QTL at FDR 10%.

** Options to explore:** 
   - Peak region definition, curretly used Yifan's way, combine samples and then call peaks.
   - covariates for samples, currently none (do we need to correct batch effects etc?)
   - GC bias correction, currently none.
   - Rasqual run option, currently follow the above paper. Used output from column 11 (chi-squared) as the test statistic. run with -r option 1000 times for each peak. 

## A VCF file with *phased* SNP genotypes and allele-specific counts.

**Liftover vcf files to ch38.**
ATAC-seq bam files are paired-end (`samtools view -f 1 xx.bam` ) and aligned to ch38 ( `samtools view -H 1 xx.bam`). The reference file were downloaded from [here](ftp://ftp-trace.ncbi.nih.gov/1000genomes/ftp/technical/reference/GRCh38_reference_genome/) and saved in `ref/GRCh38_bamref/`. The imputed and phased vcf files are too large to be converted using online tools. Therefore, use [crossmap](http://crossmap.sourceforge.net/).

In [10]:
cd /home/simingz/run_rasqual/phased_data

/project2/xinhe/simingz/run_rasqual/phased_data


In [ ]:
! source activate testother # install crossmap by conda

Download chain file from crossmap, then download GRCh38 fasta file and run crossmap. Note, though Siwei provided GRCh38 file, it still has "chr" in chromosome name. Since we only need to liftover variants in autosomes 1 to 22, we can just use hg19 to hg38 chain.

run crossmap

In [ ]:
! for i in {1..22}; do CrossMap.py vcf ../liftover_chains/hg19ToHg38.over.chain.gz chr${i}.dose.vcf.gz ../ref/GRCh38_bamref/GRCh38_full_analysis_set_plus_decoy_hla.fa chr${i}.dose.hg38.vcf;done

In [41]:
with open('chr_name_conv.txt','w') as out:
    for i in range(1,23):
        out.write(str(i)+'\tchr'+str(i) + '\n')

In [ ]:
! for i in `seq 1 22`; do awk '{ if($0 !~ /^#/) print "chr"$0; else if(match($0,/(##contig=<ID=)(.*)/,m)) print m[1]"chr"m[2]; else print $0 }' chr${i}.dose.hg38.vcf > chr${i}.dose.hg38withchr.vcf ;done

In [ ]:
! for i in `seq 1 22`; do vcf-sort chr${i}.dose.hg38withchr.vcf | bgzip > chr${i}.dose.hg38withchr.vcf.gz; tabix -p vcf chr${i}.dose.hg38withchr.vcf.gz; done

** Add AS info. Use the script provided by Rasqual.**
need around 1.5 hour for each chromosome.

In [7]:
cd /project2/xinhe/simingz/run_rasqual/rasqual/src/ASVCF/

/project2/xinhe/simingz/run_rasqual/rasqual/src/ASVCF


In [9]:
!export RASQUALDIR=/project2/xinhe/simingz/run_rasqual/rasqual/; make

gcc -o qcFilterBam qcFilterBam.o   -lm -lz
gcc -o pasteFiles pasteFiles.o   -lm -lz
gcc -o zpaste zpaste.o   -lm -lz
gcc -o countAS countAS.o   -lm -lz


In [32]:
iddict={}
with open("/home/simingz/run_rasqual/data/idmap.txt") as f:
    for line in f.readlines():
        iddict.update({line.strip().split('\t')[0]: line.strip().split('\t')[1]})

In [37]:
with open("/home/simingz/run_rasqual/phased_data/vcfsample_order.txt") as f:
    with open("/home/simingz/run_rasqual/datarun/bamlist.txt",'w') as out:
        for line in f.readlines():
            out.write("/project2/xinhe/ATAC-seq_10252018/CN/bamfiles/CN_"+ iddict[line.strip()] +"_WASPed.bam\n")

In [38]:
! cat /home/simingz/run_rasqual/datarun/bamlist.txt

/project2/xinhe/ATAC-seq_10252018/CN/bamfiles/CN_16_WASPed.bam
/project2/xinhe/ATAC-seq_10252018/CN/bamfiles/CN_08_WASPed.bam
/project2/xinhe/ATAC-seq_10252018/CN/bamfiles/CN_18_WASPed.bam
/project2/xinhe/ATAC-seq_10252018/CN/bamfiles/CN_19_WASPed.bam
/project2/xinhe/ATAC-seq_10252018/CN/bamfiles/CN_21_WASPed.bam
/project2/xinhe/ATAC-seq_10252018/CN/bamfiles/CN_10_WASPed.bam
/project2/xinhe/ATAC-seq_10252018/CN/bamfiles/CN_02_WASPed.bam
/project2/xinhe/ATAC-seq_10252018/CN/bamfiles/CN_20_WASPed.bam
/project2/xinhe/ATAC-seq_10252018/CN/bamfiles/CN_15_WASPed.bam
/project2/xinhe/ATAC-seq_10252018/CN/bamfiles/CN_14_WASPed.bam
/project2/xinhe/ATAC-seq_10252018/CN/bamfiles/CN_11_WASPed.bam
/project2/xinhe/ATAC-seq_10252018/CN/bamfiles/CN_12_WASPed.bam
/project2/xinhe/ATAC-seq_10252018/CN/bamfiles/CN_17_WASPed.bam
/project2/xinhe/ATAC-seq_10252018/CN/bamfiles/CN_06_WASPed.bam
/project2/xinhe/ATAC-seq_10252018/CN/bamfiles/CN_03_WASPed.bam
/project2/xinhe/ATAC-seq_10252018/CN/bamfiles/CN_09_WAS

In [ ]:
! for i in `seq 1 22`; do /project2/xinhe/simingz/run_rasqual/rasqual/src/ASVCF/createASVCF.sh paired_end ~/run_rasqual/datarun/bamli
st.txt chr${i}.dose.hg38withchr.vcf.gz chr${i}.hg38withchr.AS.vcf.gz atac; done

In [ ]:
! for i in `seq 1 22`; do tabix -p vcf chr${i}.hg38withchr.AS.vcf.gz; done

**generate SNP coordinate file**. This file is needed to generate the count files (see RunRasqual_DataPrep2.ipynb)

In [ ]:
bcftools query -f'[%CHROM,%POS,%ID\n]' chr1.hg38withchr.AS.vcf.gz > chr1.hg38withchr.AS.vcf.gz.snp
list